# <span style="color:orange;">My Own Color Jittering Animation</span>

# Step 1: 导入库与设置字体

In [17]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
import matplotlib.patches as patches
print("Libraries imported successfully!")

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei']  # 使用黑体
plt.rcParams['axes.unicode_minus'] = False    # 解决负号显示问题

Libraries imported successfully!


# Step 2: 设置图像大小和基础颜色


In [18]:
img_size = 512  # 使用512x512像素，是2的幂次方，确保网格完美对齐
base_color1 = np.array([1.0, 0.0, 0.0])  # 红色
base_color2 = np.array([0.0, 0.0, 1.0])  # 蓝色


* 我选择红色和蓝色作为基础颜色，因为它们是RGB颜色模型中的两个原色，
* 并且它们混合后会产生明显的紫色效果，容易观察颜色混合现象。

In [19]:
# 定义网格大小序列（每次翻倍）
grid_sizes = [2, 4, 8, 16, 32, 64, 128, 256]

# Step 3: 创建棋盘格函数

In [20]:
def create_checkerboard(grid_size, img_size, color1, color2):
    """
    创建一个棋盘格图像
    
    参数:
    grid_size: 每边的方格数量
    img_size: 图像尺寸（正方形）
    color1, color2: 两种基础颜色 (RGB格式, 范围0-1)
    
    返回:
    一个NumPy数组表示的图像
    """
    # 计算每个方格的大小
    square_size = img_size // grid_size
    
    # 创建一个全零的图像数组
    image = np.zeros((img_size, img_size, 3))
    
    # 遍历每个方格并设置颜色
    for i in range(grid_size):
        for j in range(grid_size):
            # 确定当前方格应该使用哪种颜色
            if (i + j) % 2 == 0:
                color = color1
            else:
                color = color2
            
            # 计算当前方格的起始和结束坐标
            start_i = i * square_size
            end_i = (i + 1) * square_size
            start_j = j * square_size
            end_j = (j + 1) * square_size
            
            # 将颜色应用到当前方格
            image[start_i:end_i, start_j:end_j] = color
    
    return image

* 通过确保图像尺寸能被网格尺寸整除，我们可以避免黑边问题。
* 每个方格的大小通过整数除法计算，确保没有小数像素。

# Step 4: 创建动画

In [21]:
# 创建图形和子图
fig, (ax_main, ax_ref) = plt.subplots(1, 2, figsize=(12, 6))
fig.tight_layout(pad=5.0)

# 初始化图像显示
initial_grid = create_checkerboard(grid_sizes[0], img_size, base_color1, base_color2)
img_display = ax_main.imshow(initial_grid)
ax_main.set_title(f"网格大小: {grid_sizes[0]}x{grid_sizes[0]}")
ax_main.axis('off')

# 设置参考面板
ax_ref.set_title("颜色参考")
ax_ref.axis('off')

# 添加基础颜色方块
rect1 = patches.Rectangle((0.1, 0.7), 0.2, 0.2, facecolor=base_color1, transform=ax_ref.transAxes)
rect2 = patches.Rectangle((0.1, 0.4), 0.2, 0.2, facecolor=base_color2, transform=ax_ref.transAxes)
ax_ref.add_patch(rect1)
ax_ref.add_patch(rect2)

# 添加文本标签
ax_ref.text(0.35, 0.8, "基础颜色 1 (红色)", transform=ax_ref.transAxes, fontsize=12)
ax_ref.text(0.35, 0.5, "基础颜色 2 (蓝色)", transform=ax_ref.transAxes, fontsize=12)

# 计算预测的混合颜色 (RGB平均值)
predicted_blend = (base_color1 + base_color2) / 2
blend_rect = patches.Rectangle((0.1, 0.1), 0.2, 0.2, facecolor=predicted_blend, transform=ax_ref.transAxes)
ax_ref.add_patch(blend_rect)
ax_ref.text(0.35, 0.2, f"预测混合颜色\nRGB: {predicted_blend}", transform=ax_ref.transAxes, fontsize=12)

# 动画更新函数
def update(frame):
    grid_size = grid_sizes[frame]
    checkerboard = create_checkerboard(grid_size, img_size, base_color1, base_color2)
    img_display.set_array(checkerboard)
    ax_main.set_title(f"网格大小: {grid_size}x{grid_size}")
    return img_display,

# 创建动画
animation = FuncAnimation(fig, update, frames=len(grid_sizes), 
                          interval=1000, blit=True, repeat=True)
plt.close()  # 防止显示静态图

# Step 5: 显示动画

In [23]:
# 尝试生成HTML5视频，如果失败则使用JavaScript动画
try:
    html_output = HTML(animation.to_html5_video())
except RuntimeError:
    print("HTML5 not available, using JavaScript animation instead.")
    html_output = HTML(animation.to_jshtml())

html_output

# 颜色抖动动画实践学习心得

本次通过Python实现颜色抖动动画，让我对数字图像的颜色原理有了更深刻的理解。

1. **从理论到实践的跨越**：
  * 之前仅知道RGB色彩模型的概念，通过亲手编写棋盘格生成函数，我直观地看到了红蓝两种原色如何通过空间混合
    <br>（即抖动）在人眼中形成新的紫色。这完美印证了**加色混合**原理，揭示了显示器生成百万色彩的底层逻辑——并
    <br>非直接渲染所有颜色，而是利用人眼的视觉暂留和空间整合效应。

2. **技术细节的打磨**：
  * 在实现过程中，我遇到了两个关键挑战:
    - 一是确保网格精确对齐以避免“黑边”，通过`整数除法`计算方格尺寸并选择`2的幂`作为画布大小解决了此问题。
    - 二是配置动画输出环境，最终选用`to_html5_video()`方法成功渲染出流畅的自动播放动画。

3. **收获与启示**：
  * 这项作业不仅锻炼了我的`NumPy`数组操作和`Matplotlib`动画制作能力，更重要的是培养了一种通过代码探索和验
    <br>证科学现象的思维方式。我认识到，强大的可视化工具能将抽象的理论转化为直观的感受，是学习和研究中的利器。